In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sn

In [2]:
data = pd.read_excel('/content/AvalancheTimeseriesData.xlsx')

In [3]:
df = pd.DataFrame(data)
df.head()

,Date,Wind speed,Snowfall rate,Snowpack depth,Temperature,Density of snow,Snowpack type,Surface temperature,Precipitation,Atm pressure,Relative humidity,Wind direction,Solar radiation,Snowpack stratigraphy,Terrain roughness,Snowpack wetness,Slope aspect variability,Warning
0,2023-01-08 00:00:00,10,1,100,-10,0.5,Dry,-15,0,500,50,W,1000,10 layers,Low,0.10,High,Low
1,2023-02-08 00:00:00,12,2,110,-12,0.6,Dry,-17,0,520,48,W,950,11 layers,Low,0.12,High,Low
2,2023-03-08 00:00:00,14,3,120,-14,0.7,Dry,-19,0,540,46,W,900,12 layers,Low,0.14,High,Low
3,2023-04-08 00:00:00,16,4,130,-16,0.8,Dry,-21,0,560,44,W,850,13 layers,Low,0.16,High,Low
4,2023-05-08 00:00:00,18,5,140,-18,0.9,Dry,-23,0,580,42,W,800,14 layers,Low,0.18,High,Low


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 18 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Date                      50 non-null     object 
 1   Wind speed                50 non-null     int64  
 2   Snowfall rate             50 non-null     int64  
 3   Snowpack depth            50 non-null     int64  
 4   Temperature               50 non-null     int64  
 5   Density of snow           50 non-null     float64
 6   Snowpack type             50 non-null     object 
 7   Surface temperature       50 non-null     int64  
 8   Precipitation             50 non-null     int64  
 9   Atm pressure              50 non-null     int64  
 10  Relative humidity         50 non-null     int64  
 11  Wind direction            50 non-null     object 
 12  Solar radiation           50 non-null     int64  
 13  Snowpack stratigraphy     50 non-null     object 
 14  Terrain roug

In [5]:
df.set_index('Date', inplace=True)

In [6]:
df.head()

,Wind speed,Snowfall rate,Snowpack depth,Temperature,Density of snow,Snowpack type,Surface temperature,Precipitation,Atm pressure,Relative humidity,Wind direction,Solar radiation,Snowpack stratigraphy,Terrain roughness,Snowpack wetness,Slope aspect variability,Warning
Date,,,,,,,,,,,,,,,,,
2023-01-08 00:00:00,10,1,100,-10,0.5,Dry,-15,0,500,50,W,1000,10 layers,Low,0.10,High,Low
2023-02-08 00:00:00,12,2,110,-12,0.6,Dry,-17,0,520,48,W,950,11 layers,Low,0.12,High,Low
2023-03-08 00:00:00,14,3,120,-14,0.7,Dry,-19,0,540,46,W,900,12 layers,Low,0.14,High,Low
2023-04-08 00:00:00,16,4,130,-16,0.8,Dry,-21,0,560,44,W,850,13 layers,Low,0.16,High,Low
2023-05-08 00:00:00,18,5,140,-18,0.9,Dry,-23,0,580,42,W,800,14 layers,Low,0.18,High,Low


In [7]:
df.columns

Index(['Wind speed', 'Snowfall rate', 'Snowpack depth', 'Temperature',
       'Density of snow', 'Snowpack type', 'Surface temperature',
       'Precipitation', 'Atm pressure', 'Relative humidity', 'Wind direction',
       'Solar radiation', 'Snowpack stratigraphy', 'Terrain roughness',
       'Snowpack wetness', 'Slope aspect variability', 'Warning'],
      dtype='object')

In [8]:
df = df.drop("Warning", axis=1)

In [9]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.layers import subtract, add
from sklearn.preprocessing import LabelEncoder

In [10]:
class NBeatsBlock(tf.keras.layers.Layer):
  def __init__(self,
               input_size: int,
               theta_size: int,
               horizon: int,
               n_neurons: int,
               n_layers: int,
               **kwargs):
    super().__init__(**kwargs)
    self.input_size = input_size
    self.theta_size = theta_size
    self.horizon = horizon
    self.n_neurons = n_neurons
    self.n_layers = n_layers

    self.hidden = [tf.keras.layers.Dense(n_neurons, activation="relu") for _ in range(n_layers)]
    self.theta_layer = tf.keras.layers.Dense(theta_size, activation="linear", name="theta")

  def call(self, inputs):
    x = inputs
    for layer in self.hidden:
      x = layer(x)
    theta = self.theta_layer(x)
    backcast, forecast = theta[:, :self.input_size], theta[:, -self.horizon:]
    return backcast, forecast

In [11]:
HORIZON = 1
WINDOW_SIZE = 7

In [12]:
df.columns

Index(['Wind speed', 'Snowfall rate', 'Snowpack depth', 'Temperature',
       'Density of snow', 'Snowpack type', 'Surface temperature',
       'Precipitation', 'Atm pressure', 'Relative humidity', 'Wind direction',
       'Solar radiation', 'Snowpack stratigraphy', 'Terrain roughness',
       'Snowpack wetness', 'Slope aspect variability'],
      dtype='object')

In [15]:
def function(df):
  for column_name in list(df.columns):
    column_data_type = df[column_name].dtype
    if column_data_type == "object":
      label_encoder = LabelEncoder()
      df[column_name] = label_encoder.fit_transform(df[column_name])
      df[column_name]=df[column_name].astype('int')
    df1 = df[[column_name]]
    #columns_to_drop = ['Snowfall rate', 'Snowpack depth', 'Temperature',
       #'Density of snow', 'Snowpack type', 'Surface temperature',
       #'Precipitation', 'Atm pressure', 'Relative humidity', 'Wind direction',
       #'Solar radiation', 'Snowpack stratigraphy', 'Terrain roughness',
       #'Snowpack wetness', 'Slope aspect variability']
    #df1 = df.drop(,axis = 1, inplace=True)
    #df1 = df.copy()
    for i in range(WINDOW_SIZE):
      df1[f"{column_name}+{i+1}"] = df1[column_name].shift(periods=i+1)

    # Make features and labels
    X = df1.dropna().drop(column_name, axis=1)
    y = df1.dropna()[column_name]

    # Make train and test sets
    split_size = int(len(X) * 0.8)
    X_train, y_train = X[:split_size], y[:split_size]
    X_test, y_test = X[split_size:], y[split_size:]

    # 1. Turn train and test arrays into tensor Datasets
    train_features_dataset = tf.data.Dataset.from_tensor_slices(X_train)
    train_labels_dataset = tf.data.Dataset.from_tensor_slices(y_train)

    test_features_dataset = tf.data.Dataset.from_tensor_slices(X_test)
    test_labels_dataset = tf.data.Dataset.from_tensor_slices(y_test)

    # 2. Combine features & labels
    train_dataset = tf.data.Dataset.zip((train_features_dataset, train_labels_dataset))
    test_dataset = tf.data.Dataset.zip((test_features_dataset, test_labels_dataset))

    # 3. Batch and prefetch for optimal performance
    BATCH_SIZE = 1024 # taken from Appendix D in N-BEATS paper
    train_dataset = train_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
    test_dataset = test_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

    # Values from N-BEATS paper Figure 1 and Table 18/Appendix D
    N_EPOCHS = 5000 # called "Iterations" in Table 18
    N_NEURONS = 512 # called "Width" in Table 18
    N_LAYERS = 4
    N_STACKS = 30

    INPUT_SIZE = WINDOW_SIZE * HORIZON # called "Lookback" in Table 18
    THETA_SIZE = INPUT_SIZE + HORIZON

    # Make tensors
    tensor_1 = tf.range(10) + 10
    tensor_2 = tf.range(10)

    # Subtract
    subtracted = subtract([tensor_1, tensor_2])

    # Add
    added = add([tensor_1, tensor_2])


    tf.random.set_seed(42)

    # 1. Setup N-BEATS Block layer
    nbeats_block_layer = NBeatsBlock(input_size=INPUT_SIZE,
                                 theta_size=THETA_SIZE,
                                 horizon=HORIZON,
                                 n_neurons=N_NEURONS,
                                 n_layers=N_LAYERS,
                                 name="InitialBlock")

    # 2. Create input to stacks
    stack_input = layers.Input(shape=(INPUT_SIZE), name="stack_input")

    # 3. Create initial backcast and forecast input (backwards predictions are referred to as residuals in the paper)
    backcast, forecast = nbeats_block_layer(stack_input)
    # Add in subtraction residual link, thank you to: https://github.com/mrdbourke/tensorflow-deep-learning/discussions/174
    residuals = layers.subtract([stack_input, backcast], name=f"subtract_00")
    # 4. Create stacks of blocks
    for i, _ in enumerate(range(N_STACKS-1)): # first stack is already creted in (3)

    # 5. Use the NBeatsBlock to calculate the backcast as well as block forecast
      backcast, block_forecast = NBeatsBlock(
          input_size=INPUT_SIZE,
          theta_size=THETA_SIZE,
          horizon=HORIZON,
          n_neurons=N_NEURONS,
          n_layers=N_LAYERS,
          name=f"NBeatsBlock_{i}"
        )(residuals) # pass it in residuals (the backcast)

    # 6. Create the double residual stacking
      residuals = layers.subtract([residuals, backcast], name=f"subtract_{i}")
      forecast = layers.add([forecast, block_forecast], name=f"add_{i}")

    # 7. Put the stack model together
    model = tf.keras.Model(inputs=stack_input,
                            outputs=forecast,
                            name="model_7_N-BEATS")

    # 8. Compile with MAE loss and Adam optimizer
    model.compile(loss="mae",
                  optimizer=tf.keras.optimizers.Adam(0.001),
                  metrics=["mae", "mse"])

    # 9. Fit the model with EarlyStopping and ReduceLROnPlateau callbacks
    model.fit(train_dataset,
              epochs=N_EPOCHS,
              validation_data=test_dataset,
              verbose=0, # prevent large amounts of training outputs
              # callbacks=[create_model_checkpoint(model_name=stack_model.name)] # saving model every epoch consumes far too much time
              callbacks=[tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=200, restore_best_weights=True),
                        tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss", patience=100, verbose=1)])

    # Evaluate N-BEATS model on the test dataset
    model.evaluate(test_dataset)

    def make_preds(model, input_data):
      """
      Uses model to make predictions on input_data.

     Parameters
  ----------
      model: trained model
      input_data: windowed input data (same kind of data model was trained on)

    Returns model predictions on input_data.
      """
      forecast = model.predict(input_data)
      return tf.squeeze(forecast) # return 1D array of predictions

    model_preds = make_preds(model, test_dataset)

    def mean_absolute_scaled_error(y_true, y_pred):
      """
      Implement MASE (assuming no seasonality of data).
      """
      mae = tf.reduce_mean(tf.abs(y_true - y_pred))

      # Find MAE of naive forecast (no seasonality)
      mae_naive_no_season = tf.reduce_mean(tf.abs(y_true[1:] - y_true[:-1])) # our seasonality is 1 day (hence the shifting of 1 day)

      return mae / mae_naive_no_season

    def evaluate_preds(y_true, y_pred):
      # Make sure float32 (for metric calculations)
      y_true = tf.cast(y_true, dtype=tf.float32)
      y_pred = tf.cast(y_pred, dtype=tf.float32)

      # Calculate various metrics
      mae = tf.keras.metrics.mean_absolute_error(y_true, y_pred)
      mse = tf.keras.metrics.mean_squared_error(y_true, y_pred) # puts and emphasis on outliers (all errors get squared)
      rmse = tf.sqrt(mse)
      mape = tf.keras.metrics.mean_absolute_percentage_error(y_true, y_pred)
      mase = mean_absolute_scaled_error(y_true, y_pred)

      return {"mae": mae.numpy(),
            "mse": mse.numpy(),
            "rmse": rmse.numpy(),
            "mape": mape.numpy(),
            "mase": mase.numpy()}

    model_results = evaluate_preds(y_true=y_test,
                                    y_pred=model_preds)
    model_results

In [16]:
function(df)

<ipython-input-15-cf9e47c463f7>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1[f"{column_name}+{i+1}"] = df1[column_name].shift(periods=i+1)
<ipython-input-15-cf9e47c463f7>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1[f"{column_name}+{i+1}"] = df1[column_name].shift(periods=i+1)
<ipython-input-15-cf9e47c463f7>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the document


Epoch 204: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.

Epoch 322: ReduceLROnPlateau reducing learning rate to 1.0000000474974514e-05.

Epoch 422: ReduceLROnPlateau reducing learning rate to 1.0000000656873453e-06.
1/1 [==============================] - 2s 2s/step


<ipython-input-15-cf9e47c463f7>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1[f"{column_name}+{i+1}"] = df1[column_name].shift(periods=i+1)
<ipython-input-15-cf9e47c463f7>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1[f"{column_name}+{i+1}"] = df1[column_name].shift(periods=i+1)
<ipython-input-15-cf9e47c463f7>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the document


Epoch 131: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.

Epoch 236: ReduceLROnPlateau reducing learning rate to 1.0000000474974514e-05.

Epoch 336: ReduceLROnPlateau reducing learning rate to 1.0000000656873453e-06.
1/1 [==============================] - 1s 1s/step


<ipython-input-15-cf9e47c463f7>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1[f"{column_name}+{i+1}"] = df1[column_name].shift(periods=i+1)
<ipython-input-15-cf9e47c463f7>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1[f"{column_name}+{i+1}"] = df1[column_name].shift(periods=i+1)
<ipython-input-15-cf9e47c463f7>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the document

...


Epoch 101: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.

Epoch 201: ReduceLROnPlateau reducing learning rate to 1.0000000474974514e-05.
1/1 [==============================] - 2s 2s/step


<ipython-input-15-cf9e47c463f7>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1[f"{column_name}+{i+1}"] = df1[column_name].shift(periods=i+1)
<ipython-input-15-cf9e47c463f7>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1[f"{column_name}+{i+1}"] = df1[column_name].shift(periods=i+1)
<ipython-input-15-cf9e47c463f7>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the document


Epoch 334: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.

Epoch 472: ReduceLROnPlateau reducing learning rate to 1.0000000474974514e-05.

Epoch 572: ReduceLROnPlateau reducing learning rate to 1.0000000656873453e-06.

Epoch 672: ReduceLROnPlateau reducing learning rate to 1.0000001111620805e-07.
1/1 [==============================] - 1s 1s/step


<ipython-input-15-cf9e47c463f7>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1[f"{column_name}+{i+1}"] = df1[column_name].shift(periods=i+1)
<ipython-input-15-cf9e47c463f7>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1[f"{column_name}+{i+1}"] = df1[column_name].shift(periods=i+1)
<ipython-input-15-cf9e47c463f7>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the document


Epoch 101: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.

Epoch 201: ReduceLROnPlateau reducing learning rate to 1.0000000474974514e-05.
1/1 [==============================] - 1s 1s/step
